<a href="https://colab.research.google.com/github/Sunkohli/Lab-1-Quantum-Algorithm/blob/main/DJ_algo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%pip install qiskit qiskit-ibm-runtime

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 51.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 47.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.9/363.9 kB 14.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 47.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.8/75.8 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.8/64.8 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.0/109.0 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.5/130.5 kB 7.8 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.32.3
    Uninstalling requests-2.32.3:
      Successfully uninstalled requests-2.32.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the so

In [2]:
%pip install "qiskit-aer>=0.17.0"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 82.8 MB/s eta 0:00:00


In [3]:
###DJ algo
#import required packages
import qiskit
from qiskit import QuantumRegister, QuantumCircuit
from qiskit_ibm_runtime import QiskitRuntimeService, Sampler
from qiskit import transpile
from qiskit_aer import Aer, AerSimulator
import numpy as np
from qiskit.visualization import plot_histogram

In [4]:
def dj_query(num_qubits):
    # Create a circuit implementing for a query gate for a random function
    # satisfying the promise for the Deutsch-Jozsa problem.

    qc = QuantumCircuit(num_qubits + 1)

    if np.random.randint(0, 2):
        # Flip output qubit with 50% chance
        qc.x(num_qubits)
    if np.random.randint(0, 2):
        # return constant circuit with 50% chance
        return qc

    # Choose half the possible input strings
    on_states = np.random.choice(
        range(2**num_qubits),  # numbers to sample from
        2**num_qubits // 2,  # number of samples
        replace=False,  # makes sure states are only sampled once
    )

    def add_cx(qc, bit_string):
        for qubit, bit in enumerate(reversed(bit_string)):
            if bit == "1":
                qc.x(qubit)
        return qc

    for state in on_states:
        qc.barrier()  # Barriers are added to help visualize how the functions are created.
        qc = add_cx(qc, f"{state:0b}")
        qc.mcx(list(range(num_qubits)), num_qubits)
        qc = add_cx(qc, f"{state:0b}")

    qc.barrier()

    return qc

In [5]:
def compile_circuit(function: QuantumCircuit):
    # Compiles a circuit for use in the Deutsch-Jozsa algorithm.

    n = function.num_qubits - 1
    qc = QuantumCircuit(n + 1, n)
    qc.x(n)
    qc.h(range(n + 1))
    qc.compose(function, inplace=True)
    qc.h(range(n))
    qc.measure(range(n), range(n))
    return qc

In [6]:
def dj_algorithm(function: QuantumCircuit):
    # Determine if a function is constant or balanced.
    qc = compile_circuit(function)
    backend = AerSimulator()
    qc_compiled = transpile(qc, backend)
    job_sim = backend.run(qc, shots=32,memory=True)
    result_sim = job_sim.result()
    measurements = result_sim.get_memory()
    print(measurements)
    if measurements[0] == "0":
        return "constant"
    return "balanced"


In [9]:
f = dj_query(3)
display(f.draw())
display(dj_algorithm(f))

░ ┌───┐     ┌───┐ ░       ░                 ░                 ░ 
q_0: ─░─┤ X ├──■──┤ X ├─░───■───░────────■────────░────────■────────░─
      ░ ├───┤  │  ├───┤ ░   │   ░        │        ░ ┌───┐  │  ┌───┐ ░ 
q_1: ─░─┤ X ├──■──┤ X ├─░───■───░────────■────────░─┤ X ├──■──┤ X ├─░─
      ░ ├───┤  │  ├───┤ ░   │   ░ ┌───┐  │  ┌───┐ ░ ├───┤  │  ├───┤ ░ 
q_2: ─░─┤ X ├──■──┤ X ├─░───■───░─┤ X ├──■──┤ X ├─░─┤ X ├──■──┤ X ├─░─
      ░ └───┘┌─┴─┐└───┘ ░ ┌─┴─┐ ░ └───┘┌─┴─┐└───┘ ░ └───┘┌─┴─┐└───┘ ░ 
q_3: ─░──────┤ X ├──────░─┤ X ├─░──────┤ X ├──────░──────┤ X ├──────░─
      ░      └───┘      ░ └───┘ ░      └───┘      ░      └───┘      ░

['100', '001', '110', '001', '001', '001', '100', '110', '100', '011', '100', '001', '011', '011', '011', '110', '011', '100', '110', '001', '100', '110', '011', '110', '100', '001', '100', '011', '100', '001', '100', '001']


'balanced'